# <font color="green"> Problem 1 :  COVID-19データの解析</font>

* Our World in Data の[COVID-19感染者数のページ](https://ourworldindata.org/covid-cases)から Coronavirus (COVID-19) Cases の csv データを入手せよ

In [ ]:
wget https://covid.ourworldindata.org/data/owid-covid-data.csv

* <font color="green">問:</font> 各行のセルとセルの間は何で区切られているか?

, (カンマ)

* <font color="green">問:</font> それを確認するためのコマンドを書け(ヒント: つまりカンマで区切ったときの列の個数が全行で同じならば良い)

In [ ]:
awk '{print NF}' FS=, owid-covid-data.csv | uniq

* <font color="green">問:</font> 以降のため, 列名と列番号の対応を以下のように表示するコマンドを実行せよ(ヒント: 一行目の , を改行 "\n" におきかえて表示すれば良い. それをやるコマンドは`tr`. 使い方はmanまたはググってください)

In [ ]:
head -1 owid-covid-data.csv | tr , "\n" | cat -n

* <font color="green">問:</font> locationの列に書かれている値を, 重複を排除して表示せよ (ヒント: awk, sort, uniq)

In [ ]:
awk '{print $3}' FS=, owid-covid-data.csv | sort | uniq

* <font color="green">問:</font> iso_codeの列に書かれている値を, 重複を排除して表示せよ

In [ ]:
awk '{print $1}' FS=, owid-covid-data.csv | sort | uniq

* <font color="green">問:</font> locationが国名ぽくないもの(World, Asiaなど)を排除するにはどうしたらよいか考え, それらを排除したものだけを取り出す方法を考えよ. その上で国名ぽいlocationだけを表示せよ

In [ ]:
awk '$1 !~ /OWID_/ {print $3}' FS=, owid-covid-data.csv  | sort | uniq 

* <font color="green">問:</font> それが全部で何種類あるか求めよ(それが世界の国の数(196)より多いのは国より小さい地域(United States Virgin Islandsなど)が含まれているからだと思われる. 以降はそれらを含めて「国」と呼ぶことにする

In [ ]:
awk '$1 !~ /OWID_/ {print $3}' FS=, owid-covid-data.csv  | sort | uniq | wc

* <font color="green">問:</font> 各国に対して最後にtotal_casesが報告された行だけを表示せよ(各国について, 行は日付順に並んでいると仮定して良い)

In [ ]:
awk '$1 !~ /OWID_/ && $5 != ""' FS=, owid-covid-data.csv | awk 'NR>1 && $3 != x { print last } { x=$3; last=$0; }' FS=,

* <font color="green">問:</font> 上記が各国に対する合計感染者数であると仮定して, 各国の合計感染者数を多い順に表示せよ. 自分の国は何位? 一番少ない国は? (もっともこれは報告された数であることに注意)

In [ ]:
awk '$1 !~ /OWID_/ && $5 != ""' FS=, owid-covid-data.csv | awk 'NR>1 && $3 != x { split(last,a,FS); print a[3] "," a[5] } { x=$3; last=$0; }' FS=, | sort -t , -k 2 -n -r

* <font color="green">問:</font> 一日の感染者数がもっとも多かった日は何年何月何日, どこの国で, 何人か?

In [ ]:
awk '$1 !~ /OWID_/ {print $3 FS $4 FS $6}' FS=, owid-covid-data.csv | sort -t , -k 3 -n -r | head -1

* <font color="green">問:</font> 各日付に対し, その日一日もっとも感染者(new_cases)が多かった国を求め, それを日付, 国, 感染者の順に表示せよ

In [ ]:
awk '$1 !~ /OWID_/ && $6 != ""' FS=, owid-covid-data.csv | sort -t , -k 4 | awk 'NR>1 && $4 != x { print x "," y "," z ; x=$4; y=$6; z=$3; } { if ($6 > y) { y = $6; z = $3; } }' FS=, 

# <font color="green"> Problem 2 :  Wikipedia ダンプの解析</font>

* <font color="green">問:</font> 毎回長いファイル名を打ち込まなくて良いよう, `os15_pipe` ディレクトリに `jawiki.xml`, `jawiki.xml.bz2` という名前で2つのシンボリックリンクを作れ

In [ ]:
ln -s /home/share/data/jawiki-20221220-pages-articles-multistream.xml jawiki.xml
ln -s /home/share/data/jawiki-20221220-pages-articles-multistream.xml.bz2 jawiki.xml.bz2

* <font color="green">問:</font> wikipediaのXMLデータから, 各記事のタイトル, 作者, 時刻を取り出して一行にタブ区切りで表示し, それを別のファイル (pages.csv) に保存せよ

In [ ]:
cat jawiki.xml | awk 'match($0, /<title>.*<\/title>/) {t=substr($0, RSTART+7, RLENGTH-15)} match($0, /<timestamp>.*<\/timestamp>/) {ts=substr($0, RSTART+11, RLENGTH-23)} match($0, /<username>.*<\/username>/) {u=substr($0, RSTART+10, RLENGTH-21); print t, "\t", ts, "\t", u}' > pages.csv

* <font color="green">問:</font> タイムスタンプが最古の記事はいつか? 最古な20件を表示せよ (なお, bashやzshのコマンドラインでタブを打つと補完の意味になってしまう. タブをコマンドに入力する方法はググって調べよ)

In [ ]:
sort -t $'\t' -k 2 pages.csv | head -20

* <font color="green">問:</font> 最古の記事が書かれてから現在に至るまでの, 月別の記事件数を表示せよ

In [ ]:
awk '{print $2}' FS="\t" pages.csv | awk '{printf("%s/%s\n", $1, $2)}' FS="-" | sort | uniq -c | awk '{print $2,$1}'

* <font color="green">問:</font> 最も多く記事の作者になっている人とその件数を, 上位20人表示せよ

In [ ]:
awk '{print $3}' FS="\t" pages.csv | sort | uniq -c | sort -n -r -k 1 | head -20